In [1]:
#######################################################################################################################################
######### CRIADO POR      :  MIRIAM OLIVEIRA, LAURO MIRANDA E RAFAEL OLIVEIRA                                                 #########
######### CÓDIGO ADAPTADO :  https://colab.research.google.com/drive/1GXmk1v8C-c4UtyKFqIm9HnsrVYH0pI-c                        #########
######### OBJETIVO      :  EFETUAR A LEITURA DE UMA LISTA DE LIGANTES QUE SERÃO TESTADOS PARA O RECEPTOR 5U1L                 #########
#########                  UTILIZANDO A BIBLIOTECA GNINA. OS DADOS SALVOS SERÃO EXPORTADOS PARA UM ARQUIVO DE SAÍDA           #########
######### DATA          :  24/08/2021                                                                                         #########
######### ARQUIVO DE ENTRADA : LIGANTES.XLSX       (CLICAR COM O BOTÃO DIREITO EM FILES E FAZER UPLOAD DESTE ARQUIVO)         #########
######### ARQUIVO DE SAÍDA   : GNINA_SCORE_RECEPTOR_LIGANTES.csv  (SERÁ SALVO AUTOMATICAMENTE E DISPONÍVEL POR DOWNLOAD)   #########
#######################################################################################################################################

###########################
# 1. CONFIGURAÇÃO DO AMBIENTE
###########################

# Instala Miniconda com suporte para Python 3.7
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

# Cria um novo ambiente Conda com Python 3.7 e instala rdkit e numpy
!conda create -n myenv python=3.7 numpy=1.21 rdkit -y

# Ativa o ambiente Conda
!source activate myenv

# Instala as bibliotecas necessárias
!pip install numpy==1.21.6
!pip install openpyxl
!pip install pandas==1.3.5
!pip install rdkit-pypi
!pip install xlrd

# Instala openbabel usando conda
!conda install -q -y -c conda-forge openbabel

# Instala py3Dmol
!pip install py3Dmol

# Baixa smina e gnina
!wget https://downloads.sourceforge.net/project/smina/smina.static
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina 
!chmod +x gnina

# Verifica a versão do Python e NumPy
import sys, numpy
from rdkit import Chem
print(f"Python version: {sys.version}")
print(f"NumPy version: {numpy.__version__}")
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2025-03-17 00:01:26--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolvendo repo.anaconda.com (repo.anaconda.com)... 2606:4700::6810:20f1, 2606:4700::6810:bf9e, 104.16.191.158, ...
Conectando-se a repo.anaconda.com (repo.anaconda.com)|2606:4700::6810:20f1|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 85055499 (81M) [application/x-sh]
Salvando em: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81,12M  11,0MB/s    em 7,4s    

2025-03-17 00:01:34 (11,0 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ salvo [85055499/85055499]

PREFIX=/usr/local
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: linha 392: /usr/local/conda.exe: Permissão negada
chmod: não foi possível acessar '/usr/local/conda.exe': Arquivo ou diretório inexistente
Unpacking payload ...
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: linha 404: /usr/local/conda.exe: Arquivo ou diretório inexistente
./Miniconda3-py37_4.8.2-

In [ ]:
###########################
# 2. PREPARAÇÃO DO ARQUIVO SDF DE LIGANTES
###########################

from rdkit import Chem
from rdkit.Chem import SDMolSupplier, SDWriter

# Carregar o arquivo SDF original contendo os ligantes
sdf_file = 'PubChem_5000_1.sdf'
supplier = SDMolSupplier(sdf_file)

# Criar um novo arquivo SDF com os nomes dos ligantes baseados no campo "SMILES"
output_sdf = 'sdf_com_nomes.sdf'
writer = SDWriter(output_sdf)

for idx, mol in enumerate(supplier):
    if mol is not None:
        # Se a molécula possui a propriedade "SMILES", usa seu valor; caso contrário, utiliza um nome padrão.
        if mol.HasProp("SMILES"):
            smiles_value = mol.GetProp("SMILES")
            mol.SetProp("_Name", smiles_value)
        else:
            mol.SetProp("_Name", f"Ligante_{idx + 1}")
        
        writer.write(mol)

writer.close()
print(f"Arquivo SDF com nomes baseados nos SMILES salvo como {output_sdf}")



In [2]:
#############################
#2.1 PREPARAÇÃO DO ARQUIVO SDF COM NOMES BASEADOS NO ZINC_ID
#############################

from rdkit import Chem
from rdkit.Chem import SDMolSupplier, SDWriter

# Carregar o arquivo SDF original contendo os ligantes (ex: 'molportnp.sdf')
sdf_file = 'uefsnp.sdf'
supplier = SDMolSupplier(sdf_file)

# Criar um novo arquivo SDF com os nomes dos ligantes baseados no campo "zinc_id"
output_sdf = 'sdf_com_nomes_zinc.sdf'
writer = SDWriter(output_sdf)

for idx, mol in enumerate(supplier):
    if mol is not None:
        # Se a molécula possui a propriedade "zinc_id", usa seu valor; caso contrário, utiliza um nome padrão.
        if mol.HasProp("zinc_id"):
            zinc_value = mol.GetProp("zinc_id")
        else:
            zinc_value = f"Ligante_{idx + 1}"
        mol.SetProp("_Name", zinc_value)
        writer.write(mol)
writer.close()

print(f"Arquivo SDF com nomes baseados no ZINC_ID salvo como {output_sdf}")


Arquivo SDF com nomes baseados no ZINC_ID salvo como sdf_com_nomes_zinc.sdf


In [4]:
###########################
# 3. EXECUÇÃO DO GNINA (DOCKING)
###########################

import os
import subprocess
import re
import pandas as pd
from rdkit import Chem

# Definir o caminho para o receptor (substitua conforme necessário)
receptor_file = './Maestro_hp2x7p_alphafold_trimerico.pdb'

# Arquivo SDF de ligantes (já com nomes baseados nos SMILES)
sdf_file = './Bracoli_after690_1.sdf'

# Executa o GNINA para gerar as poses e salva a saída em "ligantes_poses.sdf" e os logs em "scores.txt"
#os.system(f'./gnina -r {receptor_file} -l {sdf_file} --center_x -7.218 --center_y -44.877 --center_z -2.503 --size_x 50 --size_y 40 --size_z 40 --seed -1985 -o ligantes_poses.sdf > scores.txt 2>&1')  # ortostérico
os.system(f'./gnina -r {receptor_file} -l {sdf_file} --center_x -14.028 --center_y -36.861 --center_z -17.861 --size_x 30 --size_y 30 --size_z 30 --seed -1985 --cnn_scoring rescore -o ligantes_poses.sdf > scores.txt 2>&1')  # alostérico

###########################
# 4. PROCESSAMENTO DOS RESULTADOS (Geração do CSV)
###########################

# Lê o arquivo SDF gerado pelo GNINA (com as poses)
sdf_poses = "ligantes_poses.sdf"
supplier = Chem.SDMolSupplier(sdf_poses)
data = []

for mol in supplier:
    if mol is None:
        continue

    # Extrai o nome do ligante (armazenado na propriedade _Name)
    if mol.HasProp("_Name"):
        name = mol.GetProp("_Name")
    else:
        name = "Sem_nome"

    # Tenta ler a afinidade (GNINA usa 'affinity' ou 'minimizedAffinity')
    if mol.HasProp("affinity"):
        affinity = float(mol.GetProp("affinity"))
    elif mol.HasProp("minimizedAffinity"):
        affinity = float(mol.GetProp("minimizedAffinity"))
    else:
        affinity = None

    # Tenta ler CNNscore e CNNaffinity
    cnn_score = float(mol.GetProp("CNNscore")) if mol.HasProp("CNNscore") else None
    cnn_affinity = float(mol.GetProp("CNNaffinity")) if mol.HasProp("CNNaffinity") else None

    # Em vez da coluna "Mode", utiliza "Biblioteca" com o valor "streptome"
    biblioteca = "Bracoli"

    # Adiciona os dados para o DataFrame
    data.append([name, biblioteca, affinity, cnn_score, cnn_affinity])

# Monta o DataFrame com as colunas desejadas
df = pd.DataFrame(data, columns=["Molecule Name", "Biblioteca", "Affinity", "CNN Score", "CNN Affinity"])
df.to_csv("Ligantes_docking_results.csv", index=False)
print("CSV gerado a partir do SDF de poses!")

###########################
# 5. SELEÇÃO DA MELHOR POSE POR LIGANTE
###########################

# Lê novamente o SDF com todas as poses
docked_mols = Chem.SDMolSupplier("ligantes_poses.sdf")
best_pose_dict = {}  # chave = nome do ligante; valor = (affinity, molécula)

for mol in docked_mols:
    if mol is None:
        continue
    if mol.HasProp("_Name"):
        ligand_name = mol.GetProp("_Name")
    else:
        continue

    # Obtém a afinidade; se não existir, define como infinito
    try:
        affinity = float(mol.GetProp("affinity")) if mol.HasProp("affinity") else float('inf')
    except Exception:
        affinity = float('inf')

    # Se já houver uma pose para o ligante, mantém a de menor afinidade
    if ligand_name in best_pose_dict:
        current_best_affinity, _ = best_pose_dict[ligand_name]
        if affinity < current_best_affinity:
            best_pose_dict[ligand_name] = (affinity, mol)
    else:
        best_pose_dict[ligand_name] = (affinity, mol)

# Escreve as melhores poses em um novo arquivo SDF
writer = Chem.SDWriter("ligantes_best_poses.sdf")
for ligand, (aff, mol) in best_pose_dict.items():
    writer.write(mol)
writer.close()

print("Arquivo CSV com os resultados do docking gerado com sucesso!")
print("Arquivo SDF com as melhores poses por ligante gerado com sucesso!")
print("Processamento concluído!")


CSV gerado a partir do SDF de poses!
Arquivo CSV com os resultados do docking gerado com sucesso!
Arquivo SDF com as melhores poses por ligante gerado com sucesso!
Processamento concluído!
